# Comparison: non-ARD (Level 1) and ARD (Level 2) data

In [1]:
%matplotlib inline

import datacube
import sys
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datacube.helpers import write_geotiff

sys.path.append('../Scripts')
from deafrica_datahandling import load_ard
from deafrica_datahandling import mostcommon_crs
from deafrica_bandindices import calculate_indices
from deafrica_coastaltools import tidal_tag
from deafrica_spatialtools import subpixel_contours
from deafrica_spatialtools import interpolate_2d
from deafrica_spatialtools import contours_to_arrays
from deafrica_plotting import display_map
from deafrica_plotting import map_shapefile
from deafrica_plotting import rgb

**Connect to the datacube**

In [2]:
dc = datacube.Datacube(app='Comparison')

**Set parameters (based on Intertidal_elevation.ipynb parameters)**<p>
Should consider moving to a more built-up area as MNDWI helps distinguish between water and urban areas?

In [7]:
# Define the area of interest
lat = 11.228
lon = -15.860
buffer = 0.01

# Combine central lat,lon with buffer to get area of interest
lat_range = (lat-buffer, lat+buffer)
lon_range = (lon-buffer, lon+buffer)

# Set the range of dates for the analysis
time_range = ('2000', '2019')

# Shows map of selected area
display_map(x=lon_range, y=lat_range)

**Set queries**

In [4]:
# Create the 'query' dictionary object, which contains the longitudes, 
# latitudes and time provided above
query = {
    'x': lon_range,
    'y': lat_range,
    'time': time_range,
    'measurements': ['green','swir1'],
    'resolution': (-30, 30),
}

# Identify the most common projection system in the input query 
#output_crs = mostcommon_crs(dc=dc, product='ls8_usgs_sr_scene', query=query)

# Set the CRS as 

# Load available data from Landsat 8
landsat_ds = load_ard(dc=dc, 
                      products=['ls8_usgs_sr_scene'], 
                      output_crs=output_crs,
                      align=(15, 15),
                      group_by='solar_day',
                      **query)

Using pixel quality parameters for USGS Collection 1
Finding datasets
    ls8_usgs_sr_scene
Applying pixel quality/cloud mask
Loading 133 time steps


In [5]:
print(landsat_ds)

<xarray.Dataset>
Dimensions:      (time: 133, x: 74, y: 75)
Coordinates:
  * y            (y) float64 1.242e+06 1.242e+06 1.242e+06 ... 1.24e+06 1.24e+06
    spatial_ref  int32 32628
  * time         (time) datetime64[ns] 2013-05-03T11:23:53.310888 ... 2019-12-30T11:22:18.936330
  * x            (x) float64 4.05e+05 4.051e+05 ... 4.072e+05 4.072e+05
Data variables:
    green        (time, y, x) float32 722.0 688.0 665.0 ... 466.0 465.0 454.0
    swir1        (time, y, x) float32 1695.0 1645.0 1610.0 ... 1339.0 1353.0
Attributes:
    crs:           epsg:32628
    grid_mapping:  spatial_ref
